In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
import os
import sys

current_dir = Path.cwd()
if current_dir.name == "notebooks":
    project_root = current_dir.parent
    sys.path.insert(0, str(project_root))
else:
    project_root = Path(os.getcwd())
    sys.path.insert(0, str(project_root))

WINDOW_SIZE = 100 
TRAIN_VERSION = "v1"


agent_types = ["QLearning", "SARSA", "SARSA_Lambda", "Monte_Carlo"]
experiment_names = []

for agent in agent_types:
    experiment_names.append(f"exp_{agent.lower()}_{TRAIN_VERSION}")
    experiment_names.append(f"exp_{agent.lower()}_{TRAIN_VERSION}_finetune")


all_data = []
base_log_dir = Path(project_root) / "training_data" / "logs"

if not base_log_dir.exists():
    print(f"⚠️ Директория логов не найдена: {base_log_dir}")
    print("Убедитесь, что обучение было запущено.")
else:
    print(f"Загрузка логов из: {base_log_dir}")

for exp_name in experiment_names:
    exp_dir = base_log_dir / exp_name
    
    if not exp_dir.exists():
        print(f"  Пропуск: {exp_name} (не найден)")
        continue
    
    try:
        episodes_csv = exp_dir / "episodes.csv"
        summary_json = exp_dir / "training_summary.json"
        
        if not episodes_csv.exists():
            print(f"  Пропуск: {exp_name} (нет episodes.csv)")
            continue
            
        episodes_df = pd.read_csv(episodes_csv)
        
        if summary_json.exists():
            with open(summary_json) as f:
                summary = json.load(f)
            agent_raw_name = summary["config"]["agent_name"]
        else:
            agent_raw_name = exp_name.replace("exp_", "").replace(f"_{TRAIN_VERSION}", "").replace("_finetune", "")
        
        if "finetune" in exp_name:
            agent_label = f"{agent_raw_name.split('_')[0]} (FT)"
        else:
            agent_label = f"{agent_raw_name.split('_')[0]} (Base)"
            
        episodes_df["agent"] = agent_label
        episodes_df["experiment"] = exp_name
        all_data.append(episodes_df)
        print(f"  ✓ Загружен: {exp_name} ({len(episodes_df)} эпизодов)")
        
    except Exception as e:
        print(f"  ✗ Ошибка при загрузке {exp_name}: {e}")

if not all_data:
    print(f"\n⚠️ Не удалось загрузить данные ни одного эксперимента.")
    print(f"Проверьте:")
    print(f"  1. Путь к логам: {base_log_dir}")
    print(f"  2. TRAIN_VERSION: {TRAIN_VERSION}")
else:
    df = pd.concat(all_data, ignore_index=True)
    print(f"\n✓ Загружено {len(df['agent'].unique())} уникальных агентов из {len(all_data)} экспериментов.")


if not all_data:
    print("\nНет данных для анализа.")
else:
    final_metrics = (
        df.groupby("agent")
        .tail(WINDOW_SIZE)
        .groupby("agent")
        [["reward", "portfolio_value", "win_rate", "n_trades", "max_drawdown", "avg_pnl"]]
        .agg({
            "reward": "mean",
            "portfolio_value": "mean",
            "win_rate": "mean",
            "n_trades": "mean",
            "max_drawdown": "mean",
            "avg_pnl": "mean",
        })
    )

    final_metrics.columns = [
        "Reward (Avg)", 
        "Balance $ (Avg)", 
        "Win Rate % (Avg)", 
        "Trades (Avg)", 
        "Max DD (Avg)",
        "Avg PnL (Avg)"
    ]

    final_metrics = final_metrics.sort_values("Reward (Avg)", ascending=False)

    print(f"\n{'='*80}")
    print(f"Финальные результаты (среднее за последние {WINDOW_SIZE} эпизодов)")
    print(f"{'='*80}")
    print(final_metrics.round(2))
    print(f"{'='*80}\n")


if not all_data:
    print("Нет данных для визуализации.")
else:
    plt.style.use('seaborn-v0_8-darkgrid')
    agents = df["agent"].unique()

    fig, axes = plt.subplots(2, 2, figsize=(18, 12))

    ax1 = axes[0, 0]
    for agent in agents:
        agent_data = df[df["agent"] == agent].sort_values("episode")
        smoothed = agent_data["reward"].rolling(window=WINDOW_SIZE, min_periods=1).mean()
        ax1.plot(agent_data["episode"], smoothed, label=agent, linewidth=2)

    ax1.set_title(f"Сравнение средней награды (сглаживание {WINDOW_SIZE})", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Эпизод")
    ax1.set_ylabel("Reward (сглаженная)")
    ax1.legend(title="Агент", loc='best')
    ax1.grid(True, alpha=0.5)

    ax2 = axes[0, 1]
    for agent in agents:
        agent_data = df[df["agent"] == agent].sort_values("episode")
        smoothed_balance = agent_data["portfolio_value"].rolling(window=WINDOW_SIZE, min_periods=1).mean()
        ax2.plot(agent_data["episode"], smoothed_balance, label=agent, linewidth=2)

    INITIAL_BALANCE = 1000.0
    ax2.axhline(y=INITIAL_BALANCE, color='black', linestyle='--', alpha=0.7, label='Начальный баланс')

    ax2.set_title(f"Динамика стоимости портфеля (сглаживание {WINDOW_SIZE})", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Эпизод")
    ax2.set_ylabel("Portfolio Value ($)")
    ax2.legend(title="Агент", loc='best')
    ax2.grid(True, alpha=0.5)

    ax3 = axes[1, 0]
    for agent in agents:
        agent_data = df[df["agent"] == agent].sort_values("episode")
        smoothed_wr = agent_data["win_rate"].rolling(window=WINDOW_SIZE, min_periods=1).mean()
        ax3.plot(agent_data["episode"], smoothed_wr, label=agent, linewidth=2)

    ax3.set_title(f"Win Rate (сглаживание {WINDOW_SIZE})", fontsize=14, fontweight='bold')
    ax3.set_xlabel("Эпизод")
    ax3.set_ylabel("Win Rate (%)")
    ax3.legend(title="Агент", loc='best')
    ax3.grid(True, alpha=0.5)

    ax4 = axes[1, 1]
    for agent in agents:
        agent_data = df[df["agent"] == agent].sort_values("episode")
        smoothed_trades = agent_data["n_trades"].rolling(window=WINDOW_SIZE, min_periods=1).mean()
        ax4.plot(agent_data["episode"], smoothed_trades, label=agent, linewidth=2)

    ax4.set_title(f"Количество сделок (сглаживание {WINDOW_SIZE})", fontsize=14, fontweight='bold')
    ax4.set_xlabel("Эпизод")
    ax4.set_ylabel("Trades (сглаженное)")
    ax4.legend(title="Агент", loc='best')
    ax4.grid(True, alpha=0.5)

    plt.tight_layout()
    plt.show()

Загрузка логов из: /mnt/d/Study/Code/ml/TradingAgentClassicRL/training_data/logs
  Пропуск: exp_qlearning_v1 (не найден)
  Пропуск: exp_qlearning_v1_finetune (не найден)
  Пропуск: exp_sarsa_v1 (не найден)
  Пропуск: exp_sarsa_v1_finetune (не найден)
  Пропуск: exp_sarsa_lambda_v1 (не найден)
  Пропуск: exp_sarsa_lambda_v1_finetune (не найден)
  Пропуск: exp_monte_carlo_v1 (не найден)
  Пропуск: exp_monte_carlo_v1_finetune (не найден)

⚠️ Не удалось загрузить данные ни одного эксперимента.
Проверьте:
  1. Путь к логам: /mnt/d/Study/Code/ml/TradingAgentClassicRL/training_data/logs
  2. TRAIN_VERSION: v1
  3. Запущено ли обучение

Нет данных для анализа.
Нет данных для визуализации.
